In [1]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}

Intitializing Scala interpreter ...

Spark Web UI available at http://28d589fc47dc:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559562742838)
SparkSession available as 'spark'


import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}


### Data loading

In [17]:
// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv

val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [18]:
// combine the 3 dataframes to 1
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [19]:
// check that the rows have been aggregated correctly

println("Total data rows in loaded dataframes:"+(
    creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()))

println("Total data rows in combined dataframe:"+creditRiskdf.count())

Total data rows in loaded dataframes:183875
Total data rows in combined dataframe:183875


### Data cleansing and reshaping

In [20]:
// Drop features with inconsistent values from dataset.

val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [21]:
val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [22]:
//string array of features
val creditRiskFeatures = creditRiskdf.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [23]:
creditRiskdf.cache()

2019-06-03 12:12:30,636 WARN  [Thread-4] execution.CacheManager (Logging.scala:logWarning(66)) - Asked to cache already cached data.


res5: creditRiskdf.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [12]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
// Remove Target.

var feat_list = List[String]() //empty list to append features containing outliers in
for (feature <- creditRiskFeatures) {
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
    var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
    //check to see if thresholds are exceeded in the column and count
    var outHigh = creditRiskdf.filter(col(feature) > lit(testValHigh)).count()
    var outLow = creditRiskdf.filter(col(feature) < lit(testValLow)).count()

    //notify us whether or not column contains outliers
    if (outHigh > 0 || outLow > 0 ){
        println(feature.concat(": Contains outliers, #").concat((
            outHigh + outLow).toString))
            feat_list = feature :: feat_list
    }
    //else if (outHigh == 0 || outLow == 0){
        // println(feature.concat(": Does not contain outliers"))
    //}
}

EXT_SOURCE_2: Contains outliers, #755
client_installments_AMT_PAYMENT_min_sum: Contains outliers, #15802
AMT_CREDIT: Contains outliers, #3566
AMT_ANNUITY: Contains outliers, #3754
DAYS_EMPLOYED: Contains outliers, #43206
bureau_DAYS_CREDIT_ENDDATE_max: Contains outliers, #38077
bureau_DAYS_CREDIT_max: Contains outliers, #13523
bureau_DAYS_ENDDATE_FACT_max: Contains outliers, #10731
bureau_AMT_CREDIT_SUM_DEBT_mean: Contains outliers, #16872
previous_loans_CNT_PAYMENT_mean: Contains outliers, #7983
client_cash_CNT_INSTALMENT_FUTURE_min_max: Contains outliers, #20113
previous_loans_SELLERPLACE_AREA_max: Contains outliers, #9178
DAYS_LAST_PHONE_CHANGE: Contains outliers, #59
bureau_AMT_CREDIT_SUM_max: Contains outliers, #15707
bureau_AMT_CREDIT_SUM_mean: Contains outliers, #15604
DAYS_REGISTRATION: Contains outliers, #383
client_installments_DAYS_INSTALMENT_max_max: Contains outliers, #19805
previous_loans_AMT_DOWN_PAYMENT_max: Contains outliers, #13189
REGION_POPULATION_RELATIVE: Contains

client_cash_CNT_INSTALMENT_FUTURE_min_sum: Contains outliers, #32537
previous_loans_CHANNEL_TYPE_Contact center_count_norm: Contains outliers, #27018
bureau_CREDIT_TYPE_Car loan_count_norm: Contains outliers, #14500
client_installments_NUM_INSTALMENT_NUMBER_min_mean: Contains outliers, #16051
previous_loans_WEEKDAY_APPR_PROCESS_START_TUESDAY_count_norm: Contains outliers, #8122
previous_loans_NAME_GOODS_CATEGORY_Furniture_count_norm: Contains outliers, #26363
client_cash_NAME_CONTRACT_STATUS_Completed_count_norm_max: Contains outliers, #9884
AMT_REQ_CREDIT_BUREAU_QRT: Contains outliers, #36296
client_cash_NAME_CONTRACT_STATUS_Active_count_norm_min: Contains outliers, #13971
previous_loans_AMT_DOWN_PAYMENT_min: Contains outliers, #23887
bureau_CREDIT_TYPE_Microloan_count_norm: Contains outliers, #2557
previous_loans_DAYS_TERMINATION_min: Contains outliers, #9044
previous_loans_CHANNEL_TYPE_Channel of corporate sales_count_norm: Contains outliers, #1027
previous_loans_WEEKDAY_APPR_PROCES

previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm: Contains outliers, #6799
previous_loans_CHANNEL_TYPE_Regional / Local_count: Contains outliers, #44649
ORGANIZATION_TYPE_Transport: type 3: Contains outliers, #616
client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max: Contains outliers, #2958
ORGANIZATION_TYPE_School: Contains outliers, #5833
previous_loans_NAME_TYPE_SUITE_Other_B_count_norm: Contains outliers, #8856
bureau_CNT_CREDIT_PROLONG_mean: Contains outliers, #6567
client_bureau_balance_MONTHS_BALANCE_max_sum: Contains outliers, #45270
client_cash_NAME_CONTRACT_STATUS_Active_count_min: Contains outliers, #5538
WEEKDAY_APPR_PROCESS_START_WEDNESDAY: Contains outliers, #31521
OCCUPATION_TYPE_High skill tech staff: Contains outliers, #6918
previous_loans_NAME_PAYMENT_TYPE_XNA_count: Contains outliers, #6112
TARGET: Contains outliers, #13312


feat_list: List[String] = List(TARGET, previous_loans_NAME_PAYMENT_TYPE_XNA_count, OCCUPATION_TYPE_High skill tech staff, WEEKDAY_APPR_PROCESS_START_WEDNESDAY, client_cash_NAME_CONTRACT_STATUS_Active_count_min, client_bureau_balance_MONTHS_BALANCE_max_sum, bureau_CNT_CREDIT_PROLONG_mean, previous_loans_NAME_TYPE_SUITE_Other_B_count_norm, ORGANIZATION_TYPE_School, client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max, ORGANIZATION_TYPE_Transport: type 3, previous_loans_CHANNEL_TYPE_Regional / Local_count, previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm, client_installments_NUM_INSTALMENT_NUMBER_max_min, previous_loans_PRODUCT_COMBINATION_Card Street_count_norm, previous_loans_NAME_CONTRACT_STATUS_Refused_count, client_cash_NAME_CONTRACT_STATUS_...

In [24]:
val newlist = feat_list.filter(_!="TARGET")
val df = creditRiskdf.drop(newlist:_*)


newlist: List[String] = List(previous_loans_NAME_PAYMENT_TYPE_XNA_count, OCCUPATION_TYPE_High skill tech staff, WEEKDAY_APPR_PROCESS_START_WEDNESDAY, client_cash_NAME_CONTRACT_STATUS_Active_count_min, client_bureau_balance_MONTHS_BALANCE_max_sum, bureau_CNT_CREDIT_PROLONG_mean, previous_loans_NAME_TYPE_SUITE_Other_B_count_norm, ORGANIZATION_TYPE_School, client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max, ORGANIZATION_TYPE_Transport: type 3, previous_loans_CHANNEL_TYPE_Regional / Local_count, previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm, client_installments_NUM_INSTALMENT_NUMBER_max_min, previous_loans_PRODUCT_COMBINATION_Card Street_count_norm, previous_loans_NAME_CONTRACT_STATUS_Refused_count, client_cash_NAME_CONTRACT_STATUS_Signed_cou...

In [25]:
val creditRiskdf = df
creditRiskdf.cache()
creditRiskdf.schema

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]
res6: org.apache.spark.sql.types.StructType = StructType(StructField(EXT_SOURCE_3,StringType,true), StructField(DAYS_BIRTH,StringType,true), StructField(DAYS_ID_PUBLISH,StringType,true), StructField(CODE_GENDER_F,StringType,true), StructField(bureau_DAYS_CREDIT_min,StringType,true), StructField(previous_loans_NAME_YIELD_GROUP_high_count_norm,StringType,true), StructField(previous_loans_DAYS_LAST_DUE_1ST_VERSION_max,StringType,true), StructField(NAME_FAMILY_STATUS_Married,StringType,true), StructField(FLAG_DOCUMENT_3,StringType,true), StructField(previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm,StringType,true), StructField(client_installments_DAYS_ENTRY_PAYMENT_max_min,Str...

### Machine learning

In [26]:
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]


In [27]:
val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET"))

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_3, DAYS_BIRTH, DAYS_ID_PUBLISH, CODE_GENDER_F, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, NAME_FAMILY_STATUS_Married, FLAG_DOCUMENT_3, previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm, client_installments_DAYS_ENTRY_PAYMENT_max_min, previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm, previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm, previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm, previous_loans_NAME_PORTFOLIO_POS_count_norm, NAME_INCOME_TYPE_Working, NAME_EDUCATION_TYPE_Secondary / secondary special, previous_loans_NAME_YIELD_GROUP_middle_count_norm, previous_loans_NAME_YIELD_GROUP_XNA_count_norm, previous_loans_NAME_PRODUCT_TYPE_XNA...

In [28]:
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   train_df=train_df.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_3, DAYS_BIRTH, DAYS_ID_PUBLISH, CODE_GENDER_F, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, NAME_FAMILY_STATUS_Married, FLAG_DOCUMENT_3, previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm, client_installments_DAYS_ENTRY_PAYMENT_max_min, previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm, previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm, previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm, previous_loans_NAME_PORTFOLIO_POS_count_norm, NAME_INCOME_TYPE_Working, NAME_EDUCATION_TYPE_Secondary / secondary special, previous_loans_NAME_YIELD_GROUP_middle_count_norm, previous_loans_NAME_YIELD_GROUP_XNA_count_norm, previous_loans_NAME_PRODUCT_TYPE_XNA...

In [29]:
train_df.printSchema()

root
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- CODE_GENDER_F: float (nullable = true)
 |-- bureau_DAYS_CREDIT_min: float (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: float (nullable = true)
 |-- previous_loans_DAYS_LAST_DUE_1ST_VERSION_max: float (nullable = true)
 |-- NAME_FAMILY_STATUS_Married: float (nullable = true)
 |-- FLAG_DOCUMENT_3: float (nullable = true)
 |-- previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm: float (nullable = true)
 |-- client_installments_DAYS_ENTRY_PAYMENT_max_min: float (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm: float (nullable = true)
 |-- previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm: float (nullable = true)
 |-- previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm: float (nullable = true)
 |-- previous_loans_NAME_PORTFOLIO_POS_count_norm: float (nullable = true)
 |-- NAME_INCOME_

In [30]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val df2 = assembler.transform(train_df)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_d37c8c44ebfe
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: float, DAYS_BIRTH: float ... 29 more fields]


In [31]:
df2.printSchema()

root
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- CODE_GENDER_F: float (nullable = true)
 |-- bureau_DAYS_CREDIT_min: float (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: float (nullable = true)
 |-- previous_loans_DAYS_LAST_DUE_1ST_VERSION_max: float (nullable = true)
 |-- NAME_FAMILY_STATUS_Married: float (nullable = true)
 |-- FLAG_DOCUMENT_3: float (nullable = true)
 |-- previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm: float (nullable = true)
 |-- client_installments_DAYS_ENTRY_PAYMENT_max_min: float (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm: float (nullable = true)
 |-- previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm: float (nullable = true)
 |-- previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm: float (nullable = true)
 |-- previous_loans_NAME_PORTFOLIO_POS_count_norm: float (nullable = true)
 |-- NAME_INCOME_

In [32]:
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_0f4dadff1a38
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: float, DAYS_BIRTH: float ... 30 more fields]


In [33]:
val model = new LogisticRegression().fit(df3)
val predictions = model.transform(df3)

2019-06-03 12:13:53,029 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-06-03 12:13:53,030 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_4bdcfba66c04, numClasses = 2, numFeatures = 29
predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: float, DAYS_BIRTH: float ... 33 more fields]


In [36]:
predictions.select ("features", "label", "prediction").show(3000)

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|(29,[0,1,2,3,4,6,...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|(29,[0,1,2,3,4,5,...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|


|[0.11465249210596...|  0.0|       0.0|
|[0.11465249210596...|  0.0|       0.0|
|[0.11465249210596...|  0.0|       0.0|
|[0.11465249210596...|  0.0|       0.0|
|[0.11465249210596...|  1.0|       0.0|
|(29,[0,1,2,4,6,7,...|  0.0|       0.0|
|[0.11538666486740...|  1.0|       0.0|
|[0.11538666486740...|  0.0|       0.0|
|[0.11612491309642...|  1.0|       0.0|
|[0.11612491309642...|  0.0|       0.0|
|[0.11612491309642...|  0.0|       0.0|
|[0.11612491309642...|  1.0|       0.0|
|[0.11686726659536...|  0.0|       0.0|
|[0.11686726659536...|  0.0|       0.0|
|[0.11686726659536...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.11761373281478...|  0.0|       0.0|
|[0.11761373281478...|  0.0|       0.0|
|[0.11761373281478...|  0.0|       0.0|
|[0.11761373281478...|  1.0|       0.0|
|[0.11761373281478...|  0.0|       0.0|
|[0.11836432665586...|  0.0|       0.0|
|[0.11836432665586...|  0.0|       0.0|
|[0.11911906301975...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|


|[0.14375805854797...|  0.0|       0.0|
|[0.14375805854797...|  1.0|       0.0|
|[0.14375805854797...|  0.0|       0.0|
|[0.14375805854797...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.14375805854797...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.14464814960956...|  0.0|       0.0|
|[0.14464814960956...|  1.0|       0.0|
|[0.14464814960956...|  0.0|       0.0|
|[0.14464814960956...|  0.0|       0.0|
|[0.14464814960956...|  1.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  1.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  1.0|       0.0|
|[0.14554281532764...|  1.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|
|[0.14554281532764...|  0.0|       0.0|


|(29,[0,1,2,3,4,6,...|  1.0|       0.0|
|[0.16740845143795...|  0.0|       0.0|
|[0.16740845143795...|  0.0|       0.0|
|[0.16740845143795...|  0.0|       0.0|
|[0.16740845143795...|  0.0|       0.0|
|[0.16841617226600...|  1.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16841617226600...|  0.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|[0.16942872107028...|  1.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  1.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|
|[0.16942872107028...|  0.0|       0.0|


|[0.18959526717662...|  0.0|       0.0|
|[0.18959526717662...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.18959526717662...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  1.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  1.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  1.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|


...

|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  1.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  1.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  1.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25670558214187...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|
|[0.25808420777320...|  1.0|       0.0|
|[0.25808420777320...|  0.0|       0.0|


|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27213367819786...|  0.0|       0.0|
|[0.27356466650962...|  0.0|       0.0|
|[0.27356466650962...|  0.0|       0.0|
|[0.27356466650962...|  0.0|       0.0|
|(29,[0,1,2,4,5,6,...|  0.0|       0.0|
|[0.27356466650962...|  0.0|       0.0|
|[0.27356466650962...|  0.0|       0.0|
|[0.27356466650962...|  0.0|       0.0|


|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  1.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|(29,[0,1,2,3,4,5,...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|
|[0.30010774731636...|  0.0|       0.0|


|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|[0.31391668319702...|  0.0|       0.0|
|(29,[0,1,2,4,5,6,...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|


|[0.32647520303726...|  0.0|       0.0|
|(29,[0,1,2,3,4,5,...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  1.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  1.0|       0.0|
|[0.32647520303726...|  1.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  1.0|       0.0|
|(29,[0,1,2,3,4,5,...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|(29,[0,1,2,4,6,10...|  0.0|       0.0|
|[0.32647520303726...|  0.0|       0.0|
|[0.32806316018104...|  1.0|       0.0|
|[0.32806316018104...|  0.0|       0.0|
|[0.32806316018104...|  0.0|       0.0|
|[0.32806316018104...|  0.0|       0.0|
|[0.32806316018104...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  0.0|       0.0|
|[0.32806316018104...|  1.0|       0.0|


In [ ]:
predictions.schema